# Predictive Demand Forecasting Using Machine Learning to Reduce Food Waste
## Chapter 4: Results and Discussion

This notebook trains and evaluates **seven machine-learning models** on Nigerian food-commodity
demand data, comparing them with **four evaluation metrics** to identify the best forecasting
approach for each crop.

### Models Compared (7 models)

| # | Model | Family | Key Idea |
|---|---|---|---|
| 1 | **Random Forest** | Ensemble (bagging) | Averages many independent decision trees |
| 2 | **XGBoost** | Ensemble (boosting) | Sequentially corrects errors via gradient descent |
| 3 | **SVR** | Kernel-based | Fits an ε-insensitive tube using the RBF kernel |
| 4 | **LSTM** | Recurrent Neural Network | Memory cells with input/forget/output gates |
| 5 | **GRU** | Recurrent Neural Network | Simplified LSTM with reset & update gates |
| 6 | **Prophet** | Statistical (additive) | Decomposes series into trend + seasonality + holidays |
| 7 | **N-BEATS** | Deep learning (MLP) | Stacked FC blocks with residual learning |

### Evaluation Metrics (4 metrics)

| Metric | Full Name | Ideal |
|---|---|---|
| **RMSE** | Root Mean Squared Error | 0 (lower is better) |
| **MAE** | Mean Absolute Error | 0 (lower is better) |
| **R²** | Coefficient of Determination | 1.0 (higher is better) |
| **MAPE** | Mean Absolute Percentage Error | 0% (lower is better) |

**Strategy:** Loop through every unique commodity, train all seven models,
evaluate each with all four metrics, and identify the champion model per crop.

---


In [ ]:
# ============================================================
# Cell 1: Install Required Packages
# ============================================================
!pip install -q xgboost scikit-learn tensorflow pandas numpy matplotlib seaborn tqdm joblib prophet


In [ ]:
# ============================================================
# Cell 2: Import Libraries
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import joblib, os, json as _json, logging

# Scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# XGBoost
from xgboost import XGBRegressor

# TensorFlow / Keras (LSTM, GRU, N-BEATS)
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (LSTM, GRU, Dense, Dropout,
                                     Input, Subtract, Add)
from tensorflow.keras.callbacks import EarlyStopping

# Prophet
from prophet import Prophet
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

# Plotting settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print(f'TensorFlow version: {tf.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ============================================================
# Cell 3: Load Data
# ============================================================
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/harmonized_food_prices.csv')

print(f"\nLoaded CSV file with {len(df):,} rows and {len(df.columns)} columns.")
print(f'\nColumns: {list(df.columns)}')
print(f"\nDate range: {df['Date'].min()} to {df['Date'].max()}")
df.head(10)

## 1. Exploratory Data Analysis

> **What is EDA?**  Exploratory Data Analysis is the critical first step in any data-science project.
> Before building models, we need to *understand* the data — its shape, distributions, missing values,
> and relationships between variables.

In this section we produce two key visualisations:

| Chart | Purpose |
|---|---|
| **Top Commodities Bar Chart** | Shows which crops appear most often in the dataset. A horizontal bar chart ranks commodities by record count, revealing which ones have enough data to train reliable models. |
| **Demand Distribution Histogram** | Displays how demand values are spread. A bell-shaped (normal) distribution is ideal for regression; heavy skew might indicate the need for transformations. |


In [ ]:
# ============================================================
# Cell 4: Dataset Overview
# ============================================================
print('=' * 60)
print('DATASET OVERVIEW')
print('=' * 60)

print(f'\nShape: {df.shape}')
print(f'\nData Types:')
print(df.dtypes)
print(f'\nMissing Values:')
print(df.isnull().sum())
print(f'\nBasic Statistics:')
df.describe()

In [ ]:
# ============================================================
# Cell 5: Commodity Distribution & Demand Overview
# ============================================================
commodity_counts = df['Commodity_Name'].value_counts()
print(f'Unique Commodities: {len(commodity_counts)}\n')
print(commodity_counts.to_string())

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 15 commodities by record count
commodity_counts.head(15).plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Top 15 Commodities by Record Count')
axes[0].set_xlabel('Number of Records')

# Demand distribution
df['Demand'].hist(bins=50, ax=axes[1], color='coral', edgecolor='black')
axes[1].set_title('Distribution of Demand (Proxy)')
axes[1].set_xlabel('Demand Index (0-100)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 2. Data Preprocessing

**Why preprocess?**
Raw data often contains noise and irregularities.  Preprocessing cleans and organises the data so
machine-learning models can learn meaningful patterns instead of memorising noise.

Steps:
1. **Aggregate by Date & Commodity** — Multiple markets report prices for the same commodity on
   the same date.  We take the **mean** across all markets to create one clean time series per crop.
2. **Forward-fill missing weather data** — Weather observations may have gaps.  Forward-fill carries
   the last known value forward; back-fill covers any leading gaps.
3. **Sort chronologically** — Time-series models rely on the natural ordering of dates.


In [ ]:
# ============================================================
# Cell 6: Aggregate by Date & Commodity
# ============================================================
# Aggregate across all markets for each (Date, Commodity) pair.
# This gives us one clean time series per commodity.
df_agg = df.groupby(['Date', 'Commodity_Name']).agg({
    'Demand': 'mean',
    'Unit_Price': 'mean',        # kept for reference only, NOT used as feature
    'Avg_Temperature': 'mean',
    'Rainfall': 'mean',
    'Is_Holiday': 'max'           # 1 if any market flagged it as holiday
}).reset_index()

df_agg = df_agg.sort_values(['Commodity_Name', 'Date']).reset_index(drop=True)

# Ensure Date is datetime after groupby
df_agg['Date'] = pd.to_datetime(df_agg['Date'])

# Forward-fill missing weather data within each commodity group
df_agg['Avg_Temperature'] = df_agg.groupby('Commodity_Name')['Avg_Temperature'].transform(
    lambda x: x.ffill().bfill()
)
df_agg['Rainfall'] = df_agg.groupby('Commodity_Name')['Rainfall'].transform(
    lambda x: x.ffill().bfill()
)

print(f'Aggregated dataset: {len(df_agg):,} rows')
print(f"Unique commodities: {df_agg['Commodity_Name'].nunique()}")
print(f'\nRecords per commodity:')
print(df_agg.groupby('Commodity_Name').size().describe())
print(f'\nRemaining missing values:')
print(df_agg.isnull().sum())

## 3. Feature Engineering

> **What is Feature Engineering?**
> Raw fields alone (date, price, temperature) rarely capture the temporal *patterns* in time-series
> data.  Feature engineering creates new, more informative variables that help models detect trends,
> seasonality, and lagged effects.

### Features created

| Feature Group | Examples | Why it helps |
|---|---|---|
| **Lag features** | `Demand_Lag1`, `Lag3`, `Lag6`, `Lag12` | Capture how past demand influences future demand (autoregressive signal). |
| **Rolling means** | `RollingMean3`, `RollingMean6`, `RollingMean12` | Smooth out short-term noise and expose the underlying trend. |
| **Cyclical month encoding** | `Month_Sin`, `Month_Cos` | Encode the month as a point on a circle so that December (12) is close to January (1). |
| **Seasonal indicator** | `Is_Wet_Season` | Nigeria's wet season (April–October) strongly affects agricultural supply and, therefore, demand. |
| **Weather variables** | `Avg_Temperature`, `Rainfall` | External climate drivers of crop yield and price. |
| **Holiday flag** | `Is_Holiday` | Public holidays may disrupt market activity. |
| **Year** | `Year` | Captures long-term inflation or structural market changes. |

Creating temporal features for time-series forecasting:


In [ ]:
# ============================================================
# Cell 7: Feature Engineering Functions
# ============================================================

def create_features(group_df):
    """
    Create lag features, rolling means, and date features for a single commodity.
    Designed for monthly data (each lag unit = 1 month).
    """
    df = group_df.copy()

    # --- Lag Features (monthly data) ---
    df['Demand_Lag1']  = df['Demand'].shift(1)    # Previous month
    df['Demand_Lag3']  = df['Demand'].shift(3)    # 3 months ago
    df['Demand_Lag6']  = df['Demand'].shift(6)    # 6 months ago
    df['Demand_Lag12'] = df['Demand'].shift(12)   # Same month last year

    # --- Rolling Mean Features ---
    df['Demand_RollingMean3']  = df['Demand'].rolling(window=3).mean()    # 3-month avg
    df['Demand_RollingMean6']  = df['Demand'].rolling(window=6).mean()    # 6-month avg
    df['Demand_RollingMean12'] = df['Demand'].rolling(window=12).mean()   # 12-month avg

    # --- Date Features ---
    df['Month'] = df['Date'].dt.month
    df['Year']  = df['Date'].dt.year

    # Cyclical encoding of month (captures Jan-Dec continuity)
    df['Month_Sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_Cos'] = np.cos(2 * np.pi * df['Month'] / 12)

    # --- Nigerian Seasons ---
    # Wet season: April-October, Dry season: November-March
    df['Is_Wet_Season'] = df['Month'].apply(lambda m: 1 if 4 <= m <= 10 else 0)

    return df


# Feature columns the models will train on
FEATURE_COLS = [
    'Demand_Lag1', 'Demand_Lag3', 'Demand_Lag6', 'Demand_Lag12',
    'Demand_RollingMean3', 'Demand_RollingMean6', 'Demand_RollingMean12',
    'Avg_Temperature', 'Rainfall', 'Is_Holiday',
    'Month_Sin', 'Month_Cos', 'Is_Wet_Season', 'Year'
]

TARGET_COL = 'Demand'

# Minimum samples required to train models for a commodity
MIN_SAMPLES = 36   # At least 3 years of monthly data

print(f'Feature columns ({len(FEATURE_COLS)}): {FEATURE_COLS}')
print(f'Target: {TARGET_COL}')
print(f'Minimum samples: {MIN_SAMPLES}')

In [ ]:
# ============================================================
# Cell 8: Apply Feature Engineering to All Commodities
# ============================================================
df_features = df_agg.groupby('Commodity_Name', group_keys=False).apply(create_features)

# Check which commodities have enough data after dropping NaN rows
commodity_sample_counts = {}
for commodity in df_features['Commodity_Name'].unique():
    crop_data = df_features[df_features['Commodity_Name'] == commodity].dropna(subset=FEATURE_COLS)
    commodity_sample_counts[commodity] = len(crop_data)

valid_commodities = [c for c, n in commodity_sample_counts.items() if n >= MIN_SAMPLES]
skipped_commodities = [c for c, n in commodity_sample_counts.items() if n < MIN_SAMPLES]

print(f'Commodities with enough data (>= {MIN_SAMPLES} samples): {len(valid_commodities)}')
print(f'Commodities skipped (too few samples): {len(skipped_commodities)}')

if skipped_commodities:
    print(f'\nSkipped: {skipped_commodities}')
print(f'\nWill train models for:')
for c in valid_commodities:
    print(f'  - {c} ({commodity_sample_counts[c]} samples)')

## 4. Model Training

For each valid commodity:
1. **Prepare data**: Extract features, chronological 80/20 train/test split
2. **Train Random Forest**: 200 trees, max depth 10
3. **Train XGBoost**: 200 rounds, learning rate 0.1
4. **Train SVR**: RBF kernel, C=100 (with standard scaling)
5. **Train LSTM**: 2-layer LSTM (64 → 32 units), 12-month lookback
6. **Train GRU**: 2-layer GRU (64 → 32 units), 12-month lookback
7. **Train Prophet**: Facebook’s additive decomposition (trend + seasonality)
8. **Train N-BEATS**: Stacked FC blocks with residual learning, 12-month lookback
9. **Evaluate**: Compute RMSE, MAE, R², and MAPE for each model

### Model Descriptions

| Model | Family | How It Works | Strengths |
|---|---|---|---|
| **Random Forest** | Ensemble (bagging) | Builds many independent decision trees on random subsets of data and averages their predictions. | Robust to outliers, handles non-linear relationships, provides feature importance scores. |
| **XGBoost** | Ensemble (boosting) | Builds trees *sequentially* — each new tree corrects the errors of the previous ones using gradient descent. | Often the most accurate on tabular data; built-in regularisation. |
| **SVR** | Kernel-based | Fits a curve (via the RBF kernel) that keeps predictions within an ε-margin of the true values. | Effective in high-dimensional spaces; captures non-linear patterns via the kernel trick. |
| **LSTM** | Recurrent NN | Processes sequences step-by-step using *memory cells* with input, forget, and output gates. | Captures long-range temporal dependencies. |
| **GRU** | Recurrent NN | Simplified LSTM with only two gates (reset & update). | Fewer parameters → faster training; often matches LSTM on shorter sequences. |
| **Prophet** | Statistical | Decomposes a time series into **trend** + **seasonality** + **holidays** using a piecewise linear growth curve and Fourier series. | Handles missing data gracefully; built-in holiday support; highly interpretable components. |
| **N-BEATS** | Deep Learning (MLP) | Stacks of fully-connected blocks. Each block outputs a *backcast* (reconstructing input) and a *forecast* (predicting future) via residual learning. | Purpose-built for time-series forecasting; no RNN needed; often outperforms LSTM/GRU on benchmarks. |


In [ ]:
# ============================================================
# Cell 9: Model Helper Functions
# ============================================================

def evaluate_model(y_true, y_pred):
    """Calculate RMSE, MAE, R², and MAPE."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)

    # MAPE - avoid division by zero
    mask = y_true != 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan

    return {
        'RMSE': round(rmse, 4),
        'MAE':  round(mae, 4),
        'R²':   round(r2, 4),
        'MAPE': round(mape, 2),
    }


def build_lstm_model(n_features, lookback):
    """Build an LSTM model for time-series forecasting."""
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(lookback, n_features)),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def build_gru_model(n_features, lookback):
    """Build a GRU model for time-series forecasting."""
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(lookback, n_features)),
        Dropout(0.2),
        GRU(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def build_nbeats_model(input_dim, n_stacks=2, n_blocks=3, hidden_dim=128):
    """
    Build a simplified N-BEATS model for demand forecasting.

    N-BEATS uses stacks of fully-connected blocks.  Each block outputs a
    'backcast' (reconstructing the input) and a 'forecast' (predicting the
    target).  The residual from each block feeds the next; all forecasts
    are summed.
    """
    inputs = Input(shape=(input_dim,))
    residual = inputs
    forecast_sum = None

    for s in range(n_stacks):
        for b in range(n_blocks):
            # 4-layer FC block (following the original N-BEATS paper)
            h = Dense(hidden_dim, activation='relu')(residual)
            h = Dense(hidden_dim, activation='relu')(h)
            h = Dense(hidden_dim, activation='relu')(h)
            h = Dense(hidden_dim, activation='relu')(h)

            # Backcast: reconstruct input (residual learning)
            backcast = Dense(input_dim, activation='linear')(h)
            # Forecast: predict target
            block_forecast = Dense(1, activation='linear')(h)

            # Update residual
            residual = Subtract()([residual, backcast])

            # Accumulate forecast
            if forecast_sum is None:
                forecast_sum = block_forecast
            else:
                forecast_sum = Add()([forecast_sum, block_forecast])

    model = Model(inputs=inputs, outputs=forecast_sum)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


def create_lstm_sequences(X, y, lookback=12):
    """Create sliding-window sequences for LSTM / GRU / N-BEATS input."""
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i - lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)


print('Helper functions defined.')


In [ ]:
# ============================================================
# Cell 10: Per-Crop Training Function
# ============================================================

def train_and_evaluate_crop(commodity_name, df_features, feature_cols, target_col,
                            test_size=0.2, lstm_lookback=12):
    """
    Train RF, XGBoost, SVR, LSTM, GRU, Prophet, and N-BEATS for a single commodity.
    Returns dict with model results, predictions, test data, and trained models.
    """
    # Filter and prepare data
    crop_data = df_features[df_features['Commodity_Name'] == commodity_name].copy()
    crop_data = crop_data.dropna(subset=feature_cols + [target_col])
    crop_data = crop_data.sort_values('Date').reset_index(drop=True)

    if len(crop_data) < MIN_SAMPLES:
        return None

    # Chronological train/test split
    split_idx = int(len(crop_data) * (1 - test_size))
    train_data = crop_data.iloc[:split_idx]
    test_data  = crop_data.iloc[split_idx:]

    X_train = train_data[feature_cols].values
    y_train = train_data[target_col].values
    X_test  = test_data[feature_cols].values
    y_test  = test_data[target_col].values
    test_dates = test_data['Date'].values

    results     = {}
    predictions = {}
    trained_models = {}

    # -----------------------------------------------------------------
    # 1. RANDOM FOREST
    # -----------------------------------------------------------------
    rf_model = RandomForestRegressor(
        n_estimators=200, max_depth=10,
        min_samples_split=5, random_state=42, n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    results['Random Forest'] = evaluate_model(y_test, rf_pred)
    predictions['Random Forest'] = rf_pred
    trained_models['Random Forest'] = rf_model

    # -----------------------------------------------------------------
    # 2. XGBOOST
    # -----------------------------------------------------------------
    xgb_model = XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, verbosity=0
    )
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    results['XGBoost'] = evaluate_model(y_test, xgb_pred)
    predictions['XGBoost'] = xgb_pred
    trained_models['XGBoost'] = xgb_model

    # -----------------------------------------------------------------
    # 3. SVR (Support Vector Regression)
    # -----------------------------------------------------------------
    svr_scaler_X = StandardScaler()
    svr_scaler_y = StandardScaler()

    X_train_svr = svr_scaler_X.fit_transform(X_train)
    y_train_svr = svr_scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
    X_test_svr  = svr_scaler_X.transform(X_test)

    svr_model = SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1)
    svr_model.fit(X_train_svr, y_train_svr)
    svr_pred_scaled = svr_model.predict(X_test_svr)
    svr_pred = svr_scaler_y.inverse_transform(svr_pred_scaled.reshape(-1, 1)).ravel()

    results['SVR'] = evaluate_model(y_test, svr_pred)
    predictions['SVR'] = svr_pred
    trained_models['SVR'] = {
        'model': svr_model,
        'scaler_X': svr_scaler_X,
        'scaler_y': svr_scaler_y,
    }

    # -----------------------------------------------------------------
    # 4. LSTM
    # -----------------------------------------------------------------
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_all = crop_data[feature_cols].values
    y_all = crop_data[target_col].values.reshape(-1, 1)

    X_scaled = scaler_X.fit_transform(X_all)
    y_scaled = scaler_y.fit_transform(y_all).flatten()

    X_seq, y_seq = create_lstm_sequences(X_scaled, y_scaled, lookback=lstm_lookback)

    if len(X_seq) < MIN_SAMPLES:
        results['LSTM'] = {'RMSE': np.nan, 'MAE': np.nan, 'R²': np.nan, 'MAPE': np.nan}
        predictions['LSTM'] = np.full(len(y_test), np.nan)
    else:
        seq_split = split_idx - lstm_lookback
        X_seq_train, X_seq_test = X_seq[:seq_split], X_seq[seq_split:]
        y_seq_train, y_seq_test = y_seq[:seq_split], y_seq[seq_split:]

        lstm_model = build_lstm_model(len(feature_cols), lstm_lookback)
        early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
        lstm_model.fit(X_seq_train, y_seq_train,
                       epochs=100, batch_size=16,
                       callbacks=[early_stop], verbose=0)

        lstm_pred_scaled = lstm_model.predict(X_seq_test, verbose=0).flatten()
        lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled.reshape(-1, 1)).flatten()
        y_seq_test_inv = scaler_y.inverse_transform(y_seq_test.reshape(-1, 1)).flatten()

        results['LSTM'] = evaluate_model(y_seq_test_inv, lstm_pred)

        # Align predictions with test dates
        lstm_full = np.full(len(y_test), np.nan)
        lstm_full[:len(lstm_pred)] = lstm_pred
        predictions['LSTM'] = lstm_full
        trained_models['LSTM'] = lstm_model

    # -----------------------------------------------------------------
    # 5. GRU
    # -----------------------------------------------------------------
    if len(X_seq) < MIN_SAMPLES:
        results['GRU'] = {'RMSE': np.nan, 'MAE': np.nan, 'R²': np.nan, 'MAPE': np.nan}
        predictions['GRU'] = np.full(len(y_test), np.nan)
    else:
        gru_model = build_gru_model(len(feature_cols), lstm_lookback)
        early_stop_gru = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
        gru_model.fit(X_seq_train, y_seq_train,
                      epochs=100, batch_size=16,
                      callbacks=[early_stop_gru], verbose=0)

        gru_pred_scaled = gru_model.predict(X_seq_test, verbose=0).flatten()
        gru_pred = scaler_y.inverse_transform(gru_pred_scaled.reshape(-1, 1)).flatten()

        results['GRU'] = evaluate_model(y_seq_test_inv, gru_pred)

        gru_full = np.full(len(y_test), np.nan)
        gru_full[:len(gru_pred)] = gru_pred
        predictions['GRU'] = gru_full
        trained_models['GRU'] = gru_model

    # -----------------------------------------------------------------
    # 6. PROPHET
    # -----------------------------------------------------------------
    try:
        prophet_data = crop_data[['Date', 'Demand']].copy()
        prophet_data.columns = ['ds', 'y']
        prophet_train = prophet_data.iloc[:split_idx]
        prophet_test  = prophet_data.iloc[split_idx:]

        prophet_model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            seasonality_mode='multiplicative'
        )
        prophet_model.fit(prophet_train)

        future = prophet_test[['ds']]
        forecast = prophet_model.predict(future)
        prophet_pred = forecast['yhat'].values

        results['Prophet'] = evaluate_model(y_test, prophet_pred)
        predictions['Prophet'] = prophet_pred
        trained_models['Prophet'] = prophet_model
    except Exception as e:
        print(f"    Prophet failed: {e}")
        results['Prophet'] = {'RMSE': np.nan, 'MAE': np.nan, 'R²': np.nan, 'MAPE': np.nan}
        predictions['Prophet'] = np.full(len(y_test), np.nan)

    # -----------------------------------------------------------------
    # 7. N-BEATS
    # -----------------------------------------------------------------
    if len(X_seq) < MIN_SAMPLES:
        results['N-BEATS'] = {'RMSE': np.nan, 'MAE': np.nan, 'R²': np.nan, 'MAPE': np.nan}
        predictions['N-BEATS'] = np.full(len(y_test), np.nan)
    else:
        # Flatten sequences for N-BEATS (lookback x features -> single vector)
        X_nb_train = X_seq_train.reshape(X_seq_train.shape[0], -1)
        X_nb_test  = X_seq_test.reshape(X_seq_test.shape[0], -1)

        nbeats_model = build_nbeats_model(X_nb_train.shape[1])
        early_stop_nb = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
        nbeats_model.fit(X_nb_train, y_seq_train,
                         epochs=100, batch_size=16,
                         callbacks=[early_stop_nb], verbose=0)

        nb_pred_scaled = nbeats_model.predict(X_nb_test, verbose=0).flatten()
        nb_pred = scaler_y.inverse_transform(nb_pred_scaled.reshape(-1, 1)).flatten()

        results['N-BEATS'] = evaluate_model(y_seq_test_inv, nb_pred)

        nb_full = np.full(len(y_test), np.nan)
        nb_full[:len(nb_pred)] = nb_pred
        predictions['N-BEATS'] = nb_full
        trained_models['N-BEATS'] = nbeats_model

    # Feature importance (from Random Forest)
    fi = dict(zip(feature_cols, rf_model.feature_importances_))

    return {
        'results': results,
        'predictions': predictions,
        'y_test': y_test,
        'test_dates': test_dates,
        'train_size': len(train_data),
        'test_size': len(test_data),
        'feature_importance': fi,
        'trained_models': trained_models,
        'scalers': {'X': scaler_X, 'y': scaler_y},
    }


In [ ]:
# ============================================================
# Cell 11: Run the Multi-Crop Training Loop
# ============================================================

all_results = {}
all_crop_outputs = {}

print('=' * 70)
print('MULTI-CROP MODEL TRAINING')
print('=' * 70)

for i, commodity in enumerate(valid_commodities):
    print(f"\n{chr(9472) * 70}")
    print(f'[{i+1}/{len(valid_commodities)}] Training models for: {commodity}')
    print(f"{chr(9472) * 70}")

    output = train_and_evaluate_crop(
        commodity_name=commodity,
        df_features=df_features,
        feature_cols=FEATURE_COLS,
        target_col=TARGET_COL
    )

    if output is None:
        print(f'  Skipped (insufficient data)')
        continue

    all_crop_outputs[commodity] = output
    all_results[commodity] = output['results']

    # Print results for this commodity
    print(f"  Train: {output['train_size']} samples | Test: {output['test_size']} samples")
    for model_name, metrics in output['results'].items():
        print(f"  {model_name:15s} -> RMSE: {metrics['RMSE']:8.4f} | MAE: {metrics['MAE']:8.4f} | R²: {metrics['R²']:7.4f} | MAPE: {metrics['MAPE']:6.2f}%")

print(f"\n{'=' * 70}")
print(f'TRAINING COMPLETE: {len(all_results)} commodities processed')
print(f"{'=' * 70}")


## 5. Results & Visualisation

Now that all seven models have been trained on every valid commodity, we visualise the results to
understand which models perform best and why.

### 5.1 Actual vs. Forecast Plots

> **How to read this chart:**
> - The **black line with dots** represents the *actual* demand values from the test set.
> - Each **coloured dashed line** represents a model’s *predicted* values.
> - The closer a coloured line follows the black line, the better that model is at capturing the
>   true demand pattern.
> - Large gaps between a model’s line and the actual line indicate periods where the model struggled
>   (e.g., sudden spikes or drops in demand that the model failed to anticipate).

For each commodity, the test-set predictions from all seven models are plotted against the actual demand values.


In [ ]:
# ============================================================
# Cell 12: Actual vs. Forecast Visualisations
# ============================================================

def plot_actual_vs_forecast(commodity, output):
    """Plot actual vs predicted values for all seven models."""
    fig, ax = plt.subplots(figsize=(14, 5))

    test_dates  = pd.to_datetime(output['test_dates'])
    y_test      = output['y_test']
    predictions = output['predictions']

    # Actual values
    ax.plot(test_dates, y_test, 'k-o', label='Actual',
            linewidth=2, markersize=4, zorder=5)

    # Model predictions
    colors = {
        'Random Forest': '#2196F3',
        'XGBoost':       '#FF9800',
        'SVR':           '#9C27B0',
        'LSTM':          '#4CAF50',
        'GRU':           '#E91E63',
        'Prophet':       '#00BCD4',
        'N-BEATS':       '#FF5722',
    }
    for model_name, pred in predictions.items():
        if not np.all(np.isnan(pred)):
            ax.plot(test_dates[:len(pred)], pred, '--', label=model_name,
                    color=colors.get(model_name, 'gray'),
                    linewidth=1.5, alpha=0.85)

    ax.set_title(f'Actual vs. Forecast \u2014 {commodity}',
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Demand Index')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


# Plot for each commodity
for commodity, output in all_crop_outputs.items():
    plot_actual_vs_forecast(commodity, output)


### 5.2 Performance Comparison Table

> **Understanding the four metrics:**
>
> | Metric | Full Name | What It Measures | Ideal Value |
> |---|---|---|---|
> | **RMSE** | Root Mean Squared Error | Average prediction error, penalising large errors more heavily (because of the squaring step). | 0 |
> | **MAE** | Mean Absolute Error | Average absolute prediction error — more interpretable and less sensitive to outliers than RMSE. | 0 |
> | **R²** | Coefficient of Determination | Proportion of variance in the actual values that is explained by the model (1.0 = perfect, 0 = no better than predicting the mean, negative = worse than the mean). | 1.0 |
> | **MAPE** | Mean Absolute Percentage Error | Average error as a percentage of actual values — useful for comparing across commodities with different scales. | 0% |
>
> **How to read the heatmaps:**
> Each cell shows the metric value for one commodity–model pair.
> For RMSE, MAE, and MAPE, **green = good** (low error). For R², **green = good** (high explained variance).
> The colour gradient makes it easy to spot which model dominates.


In [ ]:
# ============================================================
# Cell 13: Performance Comparison Table & Heatmaps
# ============================================================

MODEL_ORDER = ['Random Forest', 'XGBoost', 'SVR', 'LSTM', 'GRU', 'Prophet', 'N-BEATS']

# Build comparison DataFrame
rows = []
for commodity, models in all_results.items():
    for model_name, metrics in models.items():
        rows.append({
            'Commodity': commodity,
            'Model': model_name,
            'RMSE': metrics['RMSE'],
            'MAE':  metrics['MAE'],
            'R²':   metrics['R²'],
            'MAPE (%)': metrics['MAPE'],
        })

results_df = pd.DataFrame(rows)

# Pivot tables
rmse_pivot = results_df.pivot(index='Commodity', columns='Model', values='RMSE')[MODEL_ORDER]
mae_pivot  = results_df.pivot(index='Commodity', columns='Model', values='MAE')[MODEL_ORDER]
r2_pivot   = results_df.pivot(index='Commodity', columns='Model', values='R²')[MODEL_ORDER]
mape_pivot = results_df.pivot(index='Commodity', columns='Model', values='MAPE (%)')[MODEL_ORDER]

# Print tables
for name, pivot in [('RMSE', rmse_pivot), ('MAE', mae_pivot), ('R²', r2_pivot), ('MAPE (%)', mape_pivot)]:
    direction = 'higher is better' if name == 'R²' else 'lower is better'
    print(f"\n{'=' * 70}")
    print(f'{name} COMPARISON ({direction})')
    print(f"{'=' * 70}")
    print(pivot.to_string())

# Heatmaps (2 x 2 grid)
fig, axes = plt.subplots(2, 2, figsize=(22, max(12, len(all_results) * 0.7)))

heatmap_data = [
    (rmse_pivot, 'RMSE by Commodity & Model\n(lower = better)', '.2f', 'RdYlGn_r'),
    (mae_pivot,  'MAE by Commodity & Model\n(lower = better)',  '.2f', 'RdYlGn_r'),
    (r2_pivot,   'R² by Commodity & Model\n(higher = better)',  '.3f', 'RdYlGn'),
    (mape_pivot, 'MAPE (%) by Commodity & Model\n(lower = better)', '.1f', 'RdYlGn_r'),
]

for ax, (data, title, fmt, cmap) in zip(axes.flatten(), heatmap_data):
    sns.heatmap(data, annot=True, fmt=fmt, cmap=cmap,
                ax=ax, linewidths=0.5)
    ax.set_title(title, fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()


### 5.3 Champion Model Selection

> **What is a "champion model"?**
> For each commodity, we pick the model with the **lowest RMSE** as the champion.
> RMSE is used as the primary selection criterion because it penalises large errors
> more than MAE, which is important in demand forecasting — a big prediction miss could
> lead to significant food waste or stock-outs.
>
> The **Model Win Count** summary shows how many commodities each model "won", giving
> an overall picture of which algorithm is most reliable across diverse crops.


In [ ]:
# ============================================================
# Cell 14: Champion Model per Commodity
# ============================================================

print('=' * 70)
print('CHAMPION MODEL PER COMMODITY (Lowest RMSE)')
print('=' * 70)

champion_rows = []
for commodity, models in all_results.items():
    best_model = None
    best_rmse  = float('inf')

    for model_name, metrics in models.items():
        if not np.isnan(metrics['RMSE']) and metrics['RMSE'] < best_rmse:
            best_rmse  = metrics['RMSE']
            best_model = model_name

    champion_rows.append({
        'Commodity': commodity,
        'Champion Model': best_model,
        'RMSE': best_rmse,
        'MAE':  models[best_model]['MAE']  if best_model else np.nan,
        'R²':   models[best_model]['R²']   if best_model else np.nan,
        'MAPE (%)': models[best_model]['MAPE'] if best_model else np.nan,
    })

champion_df = pd.DataFrame(champion_rows)
print(champion_df.to_string(index=False))

# Summary: which model wins most often?
print(f"\n{chr(9472) * 70}")
print('MODEL WIN COUNT:')
print(f"{chr(9472) * 70}")
win_counts = champion_df['Champion Model'].value_counts()
for model, count in win_counts.items():
    pct = count / len(champion_df) * 100
    print(f'  {model}: {count} crops ({pct:.0f}%)')

overall_best = win_counts.index[0]
print(f'\nOverall Recommended Model: {overall_best}')
print(f'  Champion for {win_counts.iloc[0]}/{len(champion_df)} commodities')


### 5.4 Feature Importance Analysis

> **What is Feature Importance?**
> Feature importance measures how much each input variable contributes to the model's predictions.
> In a **Random Forest**, importance is calculated by measuring how much each feature reduces
> prediction error (impurity) across all the trees in the forest.
>
> **How to read this chart:**
> - Features with **longer bars** are more influential — the model relies on them heavily.
> - Features with **short bars** contribute little; removing them might not hurt accuracy.
> - This helps us understand *what drives demand* (e.g., is it mostly past demand,
>   temperature, or the time of year?).
>
> We average the importance scores across all commodities to get a global view.


In [ ]:
# ============================================================
# Cell 15: Feature Importance (averaged from Random Forest)
# ============================================================

importance_dfs = []
for commodity, output in all_crop_outputs.items():
    fi = output['feature_importance']
    fi_df = pd.DataFrame([fi])
    fi_df['Commodity'] = commodity
    importance_dfs.append(fi_df)

if importance_dfs:
    all_importance = pd.concat(importance_dfs, ignore_index=True)
    avg_importance = all_importance[FEATURE_COLS].mean().sort_values(ascending=True)

    fig, ax = plt.subplots(figsize=(10, 7))
    avg_importance.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title('Average Feature Importance Across All Commodities (Random Forest)',
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('Importance')
    plt.tight_layout()
    plt.show()

    print('\nTop 5 most important features:')
    for feat, imp in avg_importance.tail(5).items():
        print(f'  {feat}: {imp:.4f}')

## 6. Export Results

This section exports:
1. **`model_comparison_results.csv`** — Full metrics (RMSE, MAE, R², MAPE) for every commodity × model combination
2. **`champion_models.csv`** — The best model per commodity with its metrics
3. **`models/` directory** — Trained champion models (`.pkl` for sklearn, `.keras` for neural nets) + metadata JSON for the web prediction UI


In [ ]:
# ============================================================
# Cell 16: Export Results
# ============================================================

# Save metric comparison tables
results_df.to_csv('model_comparison_results.csv', index=False)
champion_df.to_csv('champion_models.csv', index=False)

# -- Save trained champion models for the web prediction UI --
import pickle

os.makedirs('models', exist_ok=True)

export_data = {
    'feature_cols': FEATURE_COLS,
    'target_col': TARGET_COL,
    'commodity_stats': {},
}

for commodity, output in all_crop_outputs.items():
    champ_row = champion_df[champion_df['Commodity'] == commodity]
    if champ_row.empty:
        continue
    champ_name = champ_row.iloc[0]['Champion Model']
    model_obj  = output['trained_models'].get(champ_name)
    if model_obj is None:
        continue

    export_data['commodity_stats'][commodity] = {
        'champion_model': champ_name,
        'metrics': output['results'][champ_name],
        'train_size': output['train_size'],
        'test_size': output['test_size'],
    }

    # Save the model (sklearn models -> joblib, keras models -> keras format)
    safe_name = commodity.replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_')
    if champ_name in ('LSTM', 'GRU', 'N-BEATS'):
        model_obj.save(f'models/{safe_name}_model.keras')
    elif champ_name == 'SVR':
        joblib.dump(model_obj, f'models/{safe_name}_model.pkl')  # dict with model + scalers
    else:
        joblib.dump(model_obj, f'models/{safe_name}_model.pkl')

    # Save scalers for the commodity (needed for prediction)
    if 'scalers' in output:
        joblib.dump(output['scalers'], f'models/{safe_name}_scalers.pkl')

# Save the metadata
with open('models/metadata.json', 'w') as f:
    _json.dump(export_data, f, indent=2, default=str)

# Save all_results for the dashboard
with open('models/all_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print('Results saved:')
print('  - model_comparison_results.csv  (full metrics per commodity per model)')
print('  - champion_models.csv           (best model per commodity)')
print('  - models/                       (trained champion models + metadata)')

# Download files
from google.colab import files
import shutil

files.download('model_comparison_results.csv')
files.download('champion_models.csv')

# Zip the models folder and download it
shutil.make_archive('models', 'zip', '.', 'models')
files.download('models.zip')
